In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 67.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.2 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
model = BertForSequenceClassification.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

# Dataset
dataset = [
    ("I love BERT!", 1),
    ("BERT is great.", 1),
    ("I don't like BERT.", 0),
    ("BERT is not good.", 0)
]

# Tokenize and encode the dataset
inputs = tokenizer.batch_encode_plus(
    [data[0] for data in dataset],
    padding=True,
    truncation=True,
    return_tensors="pt"
)

# Set model to evaluation mode
model.eval()

# Forward pass
with torch.no_grad():
    outputs = model(
        input_ids=inputs['input_ids'],
        attention_mask=inputs['attention_mask']
    )

# Get predictions
predicted_labels = torch.argmax(outputs.logits, dim=1)

# Print predictions
for i, data in enumerate(dataset):
    text, label = data
    predicted_label = predicted_labels[i].item()
    print(f"Text: {text}")
    print(f"Predicted Label: {predicted_label}")
    print(f"True Label: {label}")
    print()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

Text: I love BERT!
Predicted Label: 0
True Label: 1

Text: BERT is great.
Predicted Label: 1
True Label: 1

Text: I don't like BERT.
Predicted Label: 1
True Label: 0

Text: BERT is not good.
Predicted Label: 1
True Label: 0



In [ ]:
# Python with custom data set

In [1]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 49.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 59.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 38.2 MB/s eta 0:00:00


In [2]:
from google.colab import files
uploaded = files.upload()

Saving dataset.csv to dataset.csv


In [3]:
import pandas as pd
import io

df_ = pd.read_csv(io.BytesIO(uploaded['dataset.csv']))
df = df_.copy()

df.columns = [col.upper() for col in df.columns]
df.head()

,TEXT,LABEL
0,I love using BERT for natural language process...,1
1,BERT is a powerful pre-trained model.,1
2,This sentence is unrelated to BERT.,0
3,BERT can handle various NLP tasks.,1


In [11]:
from transformers import BertTokenizer
from transformers import BertForSequenceClassification
import torch
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
from sklearn.model_selection import train_test_split

# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Load and preprocess your dataset
train_dataset = pd.read_csv(io.BytesIO(uploaded['dataset.csv']))

# Convert pandas DataFrame to PyTorch dataset
train_inputs = train_dataset['text'].tolist()
train_labels = train_dataset['label'].tolist()

# Tokenize the text and convert to input features
train_encodings = tokenizer(train_inputs, truncation=True, padding=True)

# Create PyTorch dataset
train_dataset = TensorDataset(
    torch.tensor(train_encodings['input_ids']),
    torch.tensor(train_encodings['attention_mask']),
    torch.tensor(train_labels)
)

# Split into training and validation sets
train_dataset, val_dataset = train_test_split(train_dataset, train_size=0.8, random_state=42)

# Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Load the pre-trained BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
device = torch.device('cpu')
model.to(device)

# Define optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
loss_fn = torch.nn.CrossEntropyLoss()

# Training loop
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        # Move batch to GPU
        batch = [item.to(device) for item in batch]
        input_ids, attention_mask, labels = batch

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Evaluation on the validation set
    model.eval()
    with torch.no_grad():
        total_correct = 0
        total_samples = 0

        for batch in val_loader:
            # Move batch to GPU
            batch = [item.to(device) for item in batch]
            input_ids, attention_mask, labels = batch

            # Forward pass
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=1)

            # Update evaluation metrics
            total_correct += (predictions == labels).sum().item()
            total_samples += labels.size(0)

    accuracy = total_correct / total_samples
    print(f'Epoch {epoch + 1}: Validation Accuracy = {accuracy:.4f}')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

Epoch 1: Validation Accuracy = 1.0000
Epoch 2: Validation Accuracy = 1.0000
Epoch 3: Validation Accuracy = 1.0000
Epoch 4: Validation Accuracy = 1.0000
Epoch 5: Validation Accuracy = 1.0000
